In [57]:
import pandas as pd
import numpy as np

In [58]:
covid_df = None
with open("COVID_county_data.csv") as f:
  covid_df = pd.read_csv(f)
covid_df

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0.0
1,2020-01-22,Snohomish,Washington,53061.0,1,0.0
2,2020-01-23,Snohomish,Washington,53061.0,1,0.0
3,2020-01-24,Cook,Illinois,17031.0,1,0.0
4,2020-01-24,Snohomish,Washington,53061.0,1,0.0
...,...,...,...,...,...,...
1050253,2021-02-20,Sweetwater,Wyoming,56037.0,3645,34.0
1050254,2021-02-20,Teton,Wyoming,56039.0,3318,9.0
1050255,2021-02-20,Uinta,Wyoming,56041.0,2024,12.0
1050256,2021-02-20,Washakie,Wyoming,56043.0,876,26.0


In [59]:
acs_df = None
with open("acs2017_census_tract_data.csv") as f:
  acs_df = pd.read_csv(f)
acs_df = acs_df[['State', 'County', 'TotalPop', 'Poverty', 'IncomePerCap']]

In [60]:
acs_df = acs_df[acs_df['TotalPop'] > 0]
acs_df = acs_df[acs_df['Poverty'].notna()]
acs_df = acs_df[acs_df['IncomePerCap'].notna()]

In [61]:
weighted_acs = lambda x: np.average(x, weights=acs_df.loc[x.index, "TotalPop"])

acs_df2 = pd.DataFrame(acs_df.groupby(['State', 'County']).agg(
  Population=('TotalPop','sum'),
  Poverty=('Poverty',weighted_acs),
  IncomePerCapita=('IncomePerCap',weighted_acs))).reset_index()

acs_df2

,State,County,Population,Poverty,IncomePerCapita
0,Alabama,Autauga County,55036,13.756290,27823.920161
1,Alabama,Baldwin County,203360,11.869656,29364.365583
2,Alabama,Barbour County,26201,26.867990,17561.086981
3,Alabama,Bibb County,22580,14.916032,20911.176616
4,Alabama,Blount County,57667,15.604949,22020.719562
...,...,...,...,...,...
3215,Wyoming,Sweetwater County,44527,12.079610,31699.951737
3216,Wyoming,Teton County,22923,6.842756,49200.625311
3217,Wyoming,Uinta County,20758,14.821438,27114.842808
3218,Wyoming,Washakie County,8253,12.866873,27344.988853


In [62]:
acs_df2.loc[acs_df2['County'] == 'Washington County'][acs_df2['State'] == 'Oregon']

/var/folders/y4/9tc0ndxs0n1g8nmwk290687h0000gn/T/ipykernel_8284/815958917.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  acs_df2.loc[acs_df2['County'] == 'Washington County'][acs_df2['State'] == 'Oregon']


,State,County,Population,Poverty,IncomePerCapita
2241,Oregon,Washington County,572071,10.321202,35369.047499


In [63]:
acs_df2.loc[acs_df2['County'] == 'Loudoun County'][acs_df2['State'] == 'Virginia']

/var/folders/y4/9tc0ndxs0n1g8nmwk290687h0000gn/T/ipykernel_8284/3885350199.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  acs_df2.loc[acs_df2['County'] == 'Loudoun County'][acs_df2['State'] == 'Virginia']


,State,County,Population,Poverty,IncomePerCapita
2968,Virginia,Loudoun County,374558,3.689598,50455.645745


In [64]:
acs_df2.loc[acs_df2['County'] == 'Harlan County'][acs_df2['State'] == 'Kentucky']

/var/folders/y4/9tc0ndxs0n1g8nmwk290687h0000gn/T/ipykernel_8284/3648577019.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  acs_df2.loc[acs_df2['County'] == 'Harlan County'][acs_df2['State'] == 'Kentucky']


,State,County,Population,Poverty,IncomePerCapita
1040,Kentucky,Harlan County,27548,35.669482,15456.971032


In [65]:
acs_df2.loc[acs_df2['County'] == 'Malheur County'][acs_df2['State'] == 'Oregon']

/var/folders/y4/9tc0ndxs0n1g8nmwk290687h0000gn/T/ipykernel_8284/1766506507.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  acs_df2.loc[acs_df2['County'] == 'Malheur County'][acs_df2['State'] == 'Oregon']


,State,County,Population,Poverty,IncomePerCapita
2230,Oregon,Malheur County,30421,24.298225,17567.504323


In [66]:
acs_df2.loc[acs_df2['Population'] == acs_df2['Population'].max()]

,State,County,Population,Poverty,IncomePerCapita
204,California,Los Angeles County,10082236,17.037749,30850.849058


In [67]:
acs_df2.loc[acs_df2['Population'] == acs_df2['Population'].min()]

,State,County,Population,Poverty,IncomePerCapita
2751,Texas,Loving County,74,17.1,35530.0


In [68]:
covid_df['month'] = pd.to_datetime(covid_df['date']).dt.month
covid_df['year'] = pd.to_datetime(covid_df['date']).dt.year

In [69]:
COVID_monthly = pd.DataFrame(covid_df.groupby(['state', 'county', 'month', 'year']).agg(
  Cases=('cases','sum'),
  Deaths=('deaths','sum'),
)).reset_index()

COVID_monthly

,state,county,month,year,Cases,Deaths
0,Alabama,Autauga,1,2021,156700,1773.0
1,Alabama,Autauga,2,2021,118573,1586.0
2,Alabama,Autauga,3,2020,43,0.0
3,Alabama,Autauga,4,2020,731,45.0
4,Alabama,Autauga,5,2020,3660,107.0
...,...,...,...,...,...,...
37252,Wyoming,Weston,8,2020,288,0.0
37253,Wyoming,Weston,9,2020,747,0.0
37254,Wyoming,Weston,10,2020,2730,0.0
37255,Wyoming,Weston,11,2020,9688,12.0


In [75]:
COVID_monthly.loc[COVID_monthly['county'] == 'Malheur'][COVID_monthly['state'] == 'Oregon'][COVID_monthly['month'] == 2][COVID_monthly['year'] == 2021]

/var/folders/y4/9tc0ndxs0n1g8nmwk290687h0000gn/T/ipykernel_8284/930611937.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  COVID_monthly.loc[COVID_monthly['county'] == 'Malheur'][COVID_monthly['state'] == 'Oregon'][COVID_monthly['month'] == 2][COVID_monthly['year'] == 2021]


,state,county,month,year,Cases,Deaths
25893,Oregon,Malheur,2,2021,65951,1137.0


In [71]:
COVID_monthly.loc[COVID_monthly['county'] == 'Malheur'][COVID_monthly['state'] == 'Oregon'][COVID_monthly['month'] == 8][COVID_monthly['year'] == 2020]

/var/folders/y4/9tc0ndxs0n1g8nmwk290687h0000gn/T/ipykernel_8284/993333581.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  COVID_monthly.loc[COVID_monthly['county'] == 'Malheur'][COVID_monthly['state'] == 'Oregon'][COVID_monthly['month'] == 8][COVID_monthly['year'] == 2020]


,state,county,month,year,Cases,Deaths
25899,Oregon,Malheur,8,2020,28163,459.0


In [72]:
COVID_monthly.loc[COVID_monthly['county'] == 'Malheur'][COVID_monthly['state'] == 'Oregon'][COVID_monthly['month'] == 1][COVID_monthly['year'] == 2021]

/var/folders/y4/9tc0ndxs0n1g8nmwk290687h0000gn/T/ipykernel_8284/1341186739.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  COVID_monthly.loc[COVID_monthly['county'] == 'Malheur'][COVID_monthly['state'] == 'Oregon'][COVID_monthly['month'] == 1][COVID_monthly['year'] == 2021]


,state,county,month,year,Cases,Deaths
25892,Oregon,Malheur,1,2021,96297,1627.0


In [94]:
acs_df2['County'].map(lambda x: x.removesuffix('County').strip())

0          Autauga
1          Baldwin
2          Barbour
3             Bibb
4           Blount
           ...    
3215    Sweetwater
3216         Teton
3217         Uinta
3218      Washakie
3219        Weston
Name: County, Length: 3220, dtype: object

In [101]:
pop_average = lambda x: (x / acs_df2.loc[x.index, "Population"]) * 100_000

COVID_summary = pd.concat([
  pd.Index(acs_df2.index).to_series(),
  acs_df2['State'],
  acs_df2['County'].map(lambda x: x.removesuffix('County').strip()),
  acs_df2['Population'], 
  acs_df2['Poverty'], 
  acs_df2['IncomePerCapita']],
  axis=1)

temp = pd.DataFrame(COVID_monthly.groupby(['state', 'county']).agg(
    TotalCases=('Cases','sum'),
    TotalDeaths=('Deaths','sum'),
  )).reset_index()

COVID_summary = pd.merge(COVID_summary, temp, left_on=['State', 'County'], right_on=['state', 'county'])

COVID_summary = COVID_summary[COVID_summary['Poverty'].notna()]
COVID_summary['TotalCasesPer100K'] = COVID_summary['TotalCases'] / COVID_summary['Population'] / 100_000
COVID_summary['TotalDeathsPer100K'] = COVID_summary['TotalDeaths'] / COVID_summary['Population'] / 100_000

COVID_summary

,0,State,County,Population,Poverty,IncomePerCapita,state,county,TotalCases,TotalDeaths,TotalCasesPer100K,TotalDeathsPer100K
0,0,Alabama,Autauga,55036,13.756290,27823.920161,Alabama,Autauga,645935,9042.0,0.000117,1.642925e-06
1,1,Alabama,Baldwin,203360,11.869656,29364.365583,Alabama,Baldwin,2003567,23041.0,0.000099,1.133015e-06
2,2,Alabama,Barbour,26201,26.867990,17561.086981,Alabama,Barbour,268771,4077.0,0.000103,1.556047e-06
3,3,Alabama,Bibb,22580,14.916032,20911.176616,Alabama,Bibb,261043,5272.0,0.000116,2.334810e-06
4,4,Alabama,Blount,57667,15.604949,22020.719562,Alabama,Blount,630106,8669.0,0.000109,1.503286e-06
...,...,...,...,...,...,...,...,...,...,...,...,...
3063,3215,Wyoming,Sweetwater,44527,12.079610,31699.951737,Wyoming,Sweetwater,323730,2239.0,0.000073,5.028410e-07
3064,3216,Wyoming,Teton,22923,6.842756,49200.625311,Wyoming,Teton,305376,617.0,0.000133,2.691620e-07
3065,3217,Wyoming,Uinta,20758,14.821438,27114.842808,Wyoming,Uinta,200783,1037.0,0.000097,4.995664e-07
3066,3218,Wyoming,Washakie,8253,12.866873,27344.988853,Wyoming,Washakie,84354,2622.0,0.000102,3.177027e-06


In [102]:
COVID_summary.loc[COVID_summary['County'] == 'Malheur'][COVID_summary['State'] == 'Oregon']

/var/folders/y4/9tc0ndxs0n1g8nmwk290687h0000gn/T/ipykernel_8284/221882294.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  COVID_summary.loc[COVID_summary['County'] == 'Malheur'][COVID_summary['State'] == 'Oregon']


,0,State,County,Population,Poverty,IncomePerCapita,state,county,TotalCases,TotalDeaths,TotalCasesPer100K,TotalDeathsPer100K
2156,2230,Oregon,Malheur,30421,24.298225,17567.504323,Oregon,Malheur,453634,7770.0,0.000149,0.000003


In [103]:
COVID_summary.loc[COVID_summary['County'] == 'Washington'][COVID_summary['State'] == 'Oregon']

/var/folders/y4/9tc0ndxs0n1g8nmwk290687h0000gn/T/ipykernel_8284/1477861632.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  COVID_summary.loc[COVID_summary['County'] == 'Washington'][COVID_summary['State'] == 'Oregon']


,0,State,County,Population,Poverty,IncomePerCapita,state,county,TotalCases,TotalDeaths,TotalCasesPer100K,TotalDeathsPer100K
2167,2241,Oregon,Washington,572071,10.321202,35369.047499,Oregon,Washington,2157339,22455.0,0.000038,3.925212e-07


In [104]:
COVID_summary.loc[COVID_summary['County'] == 'Loudoun'][COVID_summary['State'] == 'Virginia']

/var/folders/y4/9tc0ndxs0n1g8nmwk290687h0000gn/T/ipykernel_8284/2822057329.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  COVID_summary.loc[COVID_summary['County'] == 'Loudoun'][COVID_summary['State'] == 'Virginia']


,0,State,County,Population,Poverty,IncomePerCapita,state,county,TotalCases,TotalDeaths,TotalCasesPer100K,TotalDeathsPer100K
2816,2968,Virginia,Loudoun,374558,3.689598,50455.645745,Virginia,Loudoun,2496450,35820.0,0.000067,9.563272e-07


In [105]:
COVID_summary.loc[COVID_summary['County'] == 'Harlan'][COVID_summary['State'] == 'Kentucky']

/var/folders/y4/9tc0ndxs0n1g8nmwk290687h0000gn/T/ipykernel_8284/2117591314.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  COVID_summary.loc[COVID_summary['County'] == 'Harlan'][COVID_summary['State'] == 'Kentucky']


,0,State,County,Population,Poverty,IncomePerCapita,state,county,TotalCases,TotalDeaths,TotalCasesPer100K,TotalDeathsPer100K
1035,1040,Kentucky,Harlan,27548,35.669482,15456.971032,Kentucky,Harlan,205984,3994.0,0.000075,0.000001
